<a href="https://colab.research.google.com/github/CristianViSa/TFM/blob/main/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime
import time
import keras.backend as K
import math
import pickle
from pathlib import Path

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Bidirectional, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, roc_curve, auc, mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from numpy import concatenate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import regularizers
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [17]:
mape = tf.keras.losses.MeanAbsolutePercentageError()
def calc_mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


def calc_smape(A, F):
  A, F = np.array(A), np.array(F)
  return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


Paths y lectura de datos

In [3]:
full_data_path = "/content/drive/MyDrive/datos_web/otros/test3.csv"
ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered.csv"
# ordered_data_path = "/content/drive/MyDrive/datos_web/otros/cleaned_ordered_2.csv"
final_data_path = "/content/drive/MyDrive/datos_web/otros/final_data.csv"

# test_cleaned_full_path = "/content/drive/MyDrive/datos_web/otros/full_cleaned.csv"

In [5]:
final_data = pd.read_csv(final_data_path)
dates = pd.to_datetime(final_data["Unnamed: 0"].values)
final_data.drop("Unnamed: 0", axis = 1, inplace=True)
# Eliminar calles con datos erroneos/vacios
final_data.drop("A290", axis=1, inplace=True)
final_data.drop("A111", axis=1, inplace=True)
final_data.drop("B54", axis=1, inplace=True)
final_data.drop("A17", axis=1, inplace=True)
final_data.drop("A180", axis=1, inplace=True)
final_data.drop("A186", axis=1, inplace=True)
final_data.drop("A222", axis=1, inplace=True)
final_data.drop("A229", axis=1, inplace=True)
final_data.drop("A250", axis=1, inplace=True)
final_data.drop("A270", axis=1, inplace=True)
final_data.drop("A302", axis=1, inplace=True)
final_data.drop("A362", axis=1, inplace=True)
final_data.drop("A365", axis=1, inplace=True)
final_data.drop("A377", axis=1, inplace=True)
final_data.drop("A95", axis=1, inplace=True)
final_data.drop("B34", axis=1, inplace=True)
final_data.drop("B46", axis=1, inplace=True)
final_data.drop("B47", axis=1, inplace=True)
final_data.set_index(dates, inplace=True)


ordered_data = pd.read_csv(ordered_data_path)
ordered_data.drop("Unnamed: 0", axis = 1, inplace=True)

Variables y grupos. Seleccionar datos del conjunto

In [ ]:
barrios = {}
tipos = {}
carriles = {}
entradas = {
    "serreria" : ["A233", "A415", "A235"],
    "avcid" : ["A143", "A170", "A71"],
    "blasco" : ["A49", "A65", "A33", "A47"]
}

print(entradas)
salidas = {
    "serreria" : ["A74", "A5", "A51"],
    "avcid" : ["A72", "A245"],
    "blasco" : ["A234", "A52"]
}
# print(salidas)

# Barrios
grouped = ordered_data.groupby(ordered_data.barrio)
for name, group in grouped:
  neighs = list(set(group.ATA))
  barrios[name] = neighs
# print(barrios)

# Tipos de calle
grouped = ordered_data.groupby(ordered_data.tipo)
for name, group in grouped:
  types = list(set(group.ATA))
  tipos[name] = types
# print(tipos)

# Mismos carriles
grouped = ordered_data.groupby(ordered_data.carriles)
for name, group in grouped:
  lanes = list(set(group.ATA))
  carriles[name] = lanes
# print(carriles)

In [8]:
# Cambiar en función de la agrupacion
barrio = barrios['algiros']

datos = final_data[barrio]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos_pre = datos["2021-03-15 00:00:00":"2021-05-09 23:00:00"]
datos_post = datos["2021-05-10 00:00:00":"2021-06-20 23:00:00"]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos.fillna(0.01, inplace=True)
datos[datos == 0] = 0.01
# datos = datos.iloc[datos.index.dayofweek == 0]

# Para entrenar modelo final (NO AJUSTAR PARAMETROS CON ESTE CONJUNTO)
train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# train = datos["2021-03-15 00:00:00":"2021-05-16 23:00:00"]
val = datos["2021-05-17 00:00:00":"2021-06-06 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

mondays_train = train.iloc[train.index.dayofweek == 0]
tuesdays_train = train.iloc[train.index.dayofweek == 1]
wednesdays_train = train.iloc[train.index.dayofweek == 2]
thursdays_train = train.iloc[train.index.dayofweek == 3]
fridays_train = train.iloc[train.index.dayofweek == 4]
saturdays_train = train.iloc[train.index.dayofweek == 5]
sundays_train = train.iloc[train.index.dayofweek == 6]

mondays_test = test.iloc[test.index.dayofweek == 0]
tuesdays_test = test.iloc[test.index.dayofweek == 1]
wednesdays_test = test.iloc[test.index.dayofweek == 2]
thursdays_test = test.iloc[test.index.dayofweek == 3]
fridays_test = test.iloc[test.index.dayofweek == 4]
saturdays_test = test.iloc[test.index.dayofweek == 5]
sundays_test = test.iloc[test.index.dayofweek == 6]

features = datos.shape[1]

In [20]:
# cols = ["A71"]
# datos = pd.DataFrame(final_data[cols])
datos = final_data
# df = datos
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos_pre = datos["2021-03-15 00:00:00":"2021-05-09 23:00:00"]
datos_post = datos["2021-05-10 00:00:00":"2021-06-20 23:00:00"]
datos = datos["2021-03-15 00:00:00":"2021-06-20 23:00:00"]
datos.fillna(0.01, inplace=True)
datos[datos == 0] = 0.01
# # datos = datos.iloc[datos.index.dayofweek == 0]

# Cambiar en función de la agrupacion
barrio = barrios['algiros']
# Para entrenar modelo final (NO AJUSTAR PARAMETROS CON ESTE CONJUNTO)
train = datos["2021-03-15 00:00:00":"2021-06-06 23:00:00"]
test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# train = datos["2021-03-15 00:00:00":"2021-05-16 23:00:00"]
val = datos["2021-05-17 00:00:00":"2021-06-06 23:00:00"]
# test = datos["2021-06-07 00:00:00":"2021-06-20 23:00:00"]

# mondays_train = train.iloc[train.index.dayofweek == 0]
# tuesdays_train = train.iloc[train.index.dayofweek == 1]
# wednesdays_train = train.iloc[train.index.dayofweek == 2]
# thursdays_train = train.iloc[train.index.dayofweek == 3]
# fridays_train = train.iloc[train.index.dayofweek == 4]
# saturdays_train = train.iloc[train.index.dayofweek == 5]
# sundays_train = train.iloc[train.index.dayofweek == 6]

# mondays_test = test.iloc[test.index.dayofweek == 0]
# tuesdays_test = test.iloc[test.index.dayofweek == 1]
# wednesdays_test = test.iloc[test.index.dayofweek == 2]
# thursdays_test = test.iloc[test.index.dayofweek == 3]
# fridays_test = test.iloc[test.index.dayofweek == 4]
# saturdays_test = test.iloc[test.index.dayofweek == 5]
# sundays_test = test.iloc[test.index.dayofweek == 6]

Crear la serie temporal.

2 formas diferentes :
  1 - Método create_dataset
  2 - Método series_to_supervised

In [10]:
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset) - look_back):
    a = dataset[i: (i + look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  print(len(dataY))
  return np.array(dataX), np.array(dataY)
  
# timesteps = 1
# features = datos.shape[1]
# x_train, y_train = create_dataset(train, timesteps)
# x_val, y_val = create_dataset(val, timesteps)
# x_test, y_test = create_dataset(test, timesteps)


# x_train = np.asarray(x_train).astype('float32')
# x_train = np.reshape(x_train, (x_train.shape[0], timesteps, features))
# y_train = np.asarray(y_train).astype('float32')

# x_test = np.asarray(x_test).astype('float32')
# x_test = np.reshape(x_test, (x_test.shape[0], timesteps, features))
# y_test = np.asarray(y_test).astype('float32')

# x_val = np.asarray(x_val).astype('float32')
# x_val = np.reshape(x_val, (x_val.shape[0], timesteps, features))
# y_val = np.asarray(y_val).astype('float32')

In [22]:
# # Convertir la serie temporal en datos supervisados
# Serie sin tener en cuenta informacion adicional, solo la hora anterior
def series_to_supervised(data, keys, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('%s(t-%d)' % (keys[j], i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('%s(t)' % (keys[j])) for j in range(n_vars)]
    else:
      names += [('%s(t+%d)' % (keys[j], i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(0, inplace=True)
  return agg


features = train.shape[1]
timesteps = 2
outputs = 1

train_keys = train.keys()
test_keys = test.keys()
val_keys = val.keys()

scaler = MinMaxScaler(feature_range=(0.1,1))
scaler = scaler.fit(train)
train_values = scaler.transform(train)
test_values = scaler.transform(test)
val_values = scaler.transform(val)

serie_train = series_to_supervised(train_values, train_keys, n_in=timesteps-1, n_out=2)
serie_test = series_to_supervised(test_values, test_keys,  n_in=timesteps-1, n_out=2)
serie_val = series_to_supervised(val_values, val_keys, n_in=timesteps-1, n_out=2)


x_train, y_train = serie_train.values[:, :-features], serie_train.values[:, -features:]
x_test, y_test = serie_test.values[:, :-features], serie_test.values[:, -features:]
x_val, y_val = serie_val.values[:, :-features], serie_val.values[:, -features:]

##############  Resize de los arrays ##########
x_train = np.asarray(x_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
x_val = np.asarray(x_val).astype('float32')

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_val = np.asarray(y_val).astype('float32')


n_input = timesteps * features
x_train = x_train.reshape((x_train.shape[0], n_input))
x_test = x_test.reshape((x_test.shape[0], n_input))
x_val = x_val.reshape((x_val.shape[0], n_input))


Creación del modelo

In [23]:
####### Crear el modelo  ##########
def create_model(batch, lr, epochs, show_graph):
  model = Sequential()
  model.add(Dense(100, input_dim=n_input, activation='relu', name="Input"))
  model.add(Dense(40, activation='relu', name="Hidden"))
  model.add(Dense(features, activation='linear', name="Output"))

  model.summary()
  if show_graph:
    tf.keras.utils.plot_model(model, show_shapes=True)
  return model

Entrenar los modelos. Guarda la precisión obtenida con el conjunto de validación para su posterior análisis

In [ ]:
# Entrenar los modelos

# lrs = [0.1, 0.01, 0.001, 0.0001, 0.00001]
# batches = [1, 2, 4, 8, 16, 24, 32, 64, 128, 256]
# Best configuration
lrs = [0.0001]
batches = [24]
epochs = 400
results = {}
metrics = ["mse", "mae", "mape"]
for lr in lrs:
  for batch in batches:
    for loss in metrics:
      combination = str(lr) + "-" + str(batch) + "-" + str(loss)
      model = create_model(batch, lr, epochs, True)
      model.compile(metrics=metrics, loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

      # Path for the weights
      name = "/content/drive/MyDrive/datos_web/mlp/" + combination
      path = Path(name)      
      if not path.exists():
        path.mkdir(parents=True)
      filepath = name + "/weights.best.hdf5"
      es = EarlyStopping(monitor=loss, mode='min', verbose=1, patience=2)
      checkpoint = ModelCheckpoint(filepath, monitor=loss, verbose=1, save_best_only=True, mode='min')
      callbacks_list = [checkpoint, es]
      history = model.fit(
        x_train, y_train, 
        epochs=epochs, 
        batch_size=batch, 
        validation_data=(x_val, y_val),
        callbacks=callbacks_list,
        shuffle=False,
        verbose=1)
      plt.title("MAE")
      plt.xlabel("Épocas")
      plt.ylabel("MAE")
      plt.plot(history.history['mae'], label='entrenamiento')
      plt.plot(history.history['val_mae'], label='test')
      plt.legend()
      # plt.show()
      out_file = name + "/MAE.png"
      plt.savefig(out_file)
      plt.close()

      plt.title("MSE")
      plt.xlabel("Épocas")
      plt.ylabel("MSE")
      plt.plot(history.history['mse'], label='entrenamiento')
      plt.plot(history.history['val_mse'], label='test')
      plt.legend()
      # plt.show()
      out_file = name + "/MSE.png"
      plt.savefig(out_file)
      plt.close()
      
      plt.title("MAPE")
      plt.xlabel("Épocas")
      plt.ylabel("MAPE")
      plt.plot(history.history['mape'], label='entrenamiento')
      plt.plot(history.history['val_mape'], label='validacion')
      plt.legend()
      # plt.show()
      out_file = name + "/MAPE.png"
      plt.savefig(out_file)
      plt.close()
      # Write to a file the best loss obtained
      with open("/content/drive/MyDrive/datos_web/otros/results.txt", "a") as file_pi:
        l = np.array(history.history['loss'])
        l.sort()
        out = "Results for " + combination + "\n"
        file_pi.write(out)
        file_pi.write(str(l[0]))
        file_pi.write("\n ------------------------------------------------- \n")
        file_pi.close()

Hacer predicciones


In [ ]:
# Change the path
path = "/content/drive/MyDrive/datos_web/mlp/0.0001-24-mae/weights.best.hdf5"
model.load_weights(path)

# Hacer predicciones
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)
val_predict = model.predict(x_val)

# Inver
train_predict_inversed = scaler.inverse_transform(train_predict)
test_predict_inversed = scaler.inverse_transform(test_predict)  
val_predict_inversed = scaler.inverse_transform(val_predict)  

y_test_inversed = scaler.inverse_transform(y_test)  
y_train_inversed = scaler.inverse_transform(y_train)
y_val_inversed = scaler.inverse_transform(y_val)

# train_predict_inversed = train_predict
# test_predict_inversed = test_predict
# val_predict_inversed = val_predict

# y_test_inversed = y_test
# y_train_inversed = y_train
# y_val_inversed = y_val

total_train_rmse = 0
total_train_mae = 0
total_train_mse = 0

total_test_rmse = 0
total_test_mae = 0
total_test_mse = 0
total_test_mape = 0
total_test_smape = 0
for i in range(test_predict.shape[1]):
  train_score = math.sqrt(mean_squared_error(y_train[:, i], train_predict[:,  i]))
  train_mse_score = mean_squared_error(y_train[:, i], train_predict[:,  i])
  train_mae_score = mean_absolute_error(y_train[:, i], train_predict[:,  i])
  print('Train Score: %.5f RMSE' % (train_score))
  print('Train Score: %.5f MAE' % (train_mae_score))
  print('Train Score: %.5f MSE' % (train_mse_score))
  total_train_rmse += train_score
  total_train_mae += train_mae_score
  total_train_mse += train_mse_score
  # # print(len(test_predict))
  # # print(len(y_test))
  test_score = math.sqrt(mean_squared_error(y_test[:, i], test_predict[:,  i]))
  test_mae_score = mean_absolute_error(y_test[:, i], test_predict[:,  i])
  test_mse_score = mean_squared_error(y_test[:, i], test_predict[:,  i])
  test_mape_score = calc_mape(y_test[:, i], test_predict[:,  i])
  test_smape_score = calc_smape(y_test[:, i], test_predict[:,  i])
  print('Test Score: %.5f RMSE' % (test_score))
  print('Test Score: %.5f MAE' % (test_mae_score))
  print('Test Score: %.5f MSE' % (test_mse_score))
  print('Test Score: %.5f MAPE' % (test_mape_score))
  print('Test Score: %.5f sMAPE' % (test_smape_score))
  total_test_rmse += test_score
  total_test_mae += test_mae_score
  total_test_mse += test_mse_score
  total_test_mape += test_mape_score
  total_test_smape += test_smape_score
  fig, ax = plt.subplots()
  predicted_values = test_predict_inversed[:, i]
  real_values = y_test_inversed[:, i]
  # print(predicted_values)
  horas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
           20, 21, 22, 23]
  # ax.plot(horas, real_values[22:46], "-x",color = 'red', label = 'Real')
  # ax.plot(horas, predicted_values[22:46], "-x", color = 'blue', label = 'Forecasted')
  ax.plot(real_values, color = 'red', label = 'Reales')
  ax.plot(predicted_values, color = 'blue', label = 'Predichos')
  # ax.plot(real_values, color = 'red', label = 'Reales')
  # ax.plot(predicted_values, color = 'blue', label = 'Predichos')
  # ax.set_title(train_keys[i])
  if len(ordered_data[ordered_data.ATA==train_keys[i]]) > 0:
    name = ordered_data[ordered_data.ATA==train_keys[i]].desc.iloc[0]
  else:
    name = train_keys[i]
  # if len(name) > 15:
    # name = name[0:20] +"..."
  ax.set_xlabel('Hour')
  ax.set_title(name)
  # ax.set_xticks(x)
  # ax.set_xticks
  # ax.set_xticklabels([v for v in test_hours[4:1800]])
  ax.tick_params(labelsize ='medium', pad = 5, direction = 'out', rotation = 45)
  ax.set_ylabel('Vehicles')
  ax.set_xticks(np.arange(len(horas)))
  ax.legend()
  plt.xticks(rotation='vertical')
  plt.show()
  print("ATA -------> ", train_keys[i])
  print("-------------------- ACABA AQUI, CALLE NUEVA -----------------------")
print('Total train Score: %.4f RMSE' % (total_train_rmse))
print('Total train Score: %.4f MAE' % (total_train_mae))
print('Total train Score: %.4f MSE' % (total_train_mse))

print('Total test Score: %.4f RMSE' % (total_test_rmse))
print('Total test Score: %.4f MAE' % (total_test_mae))
print('Total test Score: %.4f MSE' % (total_test_mse))
print('Total test Score: %.4f MAPE' % (total_test_mape))
print('Total test Score: %.4f sMAPE' % (total_test_smape))


print('AVG train Score: %.4f RMSE' % (total_train_rmse / features))
print('AVG train Score: %.4f MAE' % (total_train_mae / features))
print('AVG train Score: %.4f MSE' % (total_train_mse / features))


print('AVG test Score: %.4f RMSE' % (total_test_rmse / features))
print('AVG test Score: %.4f MAE' % (total_test_mae / features))
print('AVG test Score: %.4f MSE' % (total_test_mse / features))
print('AVG test Score: %.4f MAPE' % (total_test_mape / features))
print('AVG test Score: %.4f sMAPE' % (total_test_smape / features))



In [26]:
train_mse_score = mean_squared_error(y_train, train_predict)
print("TRAIN MSE", train_mse_score)
train_mae_score = mean_absolute_error(y_train, train_predict)
print("TRAIN MAE", train_mae_score)
train_rmse_score = math.sqrt(mean_squared_error(y_train, train_predict))
print("TRAIN RMSE", train_rmse_score)

val_mse_score = mean_squared_error(y_val, val_predict)
print("VAL MSE", val_mse_score)
val_mae_score = mean_absolute_error(y_val, val_predict)
print("VAL MAE", val_mae_score)
val_rmse_score = math.sqrt(mean_squared_error(y_val, val_predict))
print("VAL RMSE", val_rmse_score)

train_mse_score = mean_squared_error(y_test, test_predict)
print("TEST MSE", test_mse_score)
train_mae_score = mean_absolute_error(y_test, test_predict)
print("TEST MAE", test_mae_score)
test_rmse_score = math.sqrt(mean_squared_error(y_test, test_predict))
print("TEST RMSE", test_rmse_score)
test_mape_score = mape(y_test, test_predict)
print("TEST MAPE", test_mape_score)




TRAIN MSE 0.009449562
TRAIN MAE 0.062286764
TRAIN RMSE 0.09720885791867295
VAL MSE 0.0106085185
VAL MAE 0.06663623
VAL RMSE 0.10299766257617567
TEST MSE 0.009413776
TEST MAE 0.06800731
TEST RMSE 0.11872667958883296
TEST MAPE tf.Tensor(21.684553, shape=(), dtype=float32)
